<a href="https://colab.research.google.com/github/20KMJ/ESAA7/blob/main/%EC%98%81%ED%99%94%EA%B4%80%EA%B0%9D%EC%88%98%EC%98%88%EC%B8%A1_%EC%B5%9C%EC%A2%85_%EC%A0%9C%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

In [31]:
train = pd.read_csv("movies_train.csv")
test = pd.read_csv("movies_test.csv")

In [32]:
# train, test 합치기
train['from'] = 'train'
test['from'] = 'test'
total = train.append(test)
total

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,from
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398.0,train
1,내부자들,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501.0,train
2,은밀하게 위대하게,(주)쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.25,4,343,4,6959083.0,train
3,나는 공무원이다,(주)NEW,코미디,2012-07-12,101,전체 관람가,구자홍,23894.00,2,20,6,217866.0,train
4,불량남녀,쇼박스(주)미디어플렉스,코미디,2010-11-04,108,15세 관람가,신근호,1.00,1,251,2,483387.0,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,해에게서 소년에게,디씨드,드라마,2015-11-19,78,15세 관람가,안슬기,2590.00,1,4,4,NaN,test
239,울보 권투부,인디스토리,다큐멘터리,2015-10-29,86,12세 관람가,이일하,NaN,0,18,2,NaN,test
240,어떤살인,(주)컨텐츠온미디어,느와르,2015-10-28,107,청소년 관람불가,안용훈,NaN,0,224,4,NaN,test
241,말하지 못한 비밀,(주)씨타마운틴픽쳐스,드라마,2015-10-22,102,청소년 관람불가,송동윤,50699.00,1,68,7,NaN,test


In [33]:
# 개봉날짜 형변환 또는 연월 분리
total['year'] = total['release_time'].apply(lambda x: int(x[:4]))
total['month'] = total['release_time'].apply(lambda x: int(x[5:7]))
total =  total.drop(['release_time'],axis = 1)
total.head(3)

,title,distributor,genre,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,from,year,month
0,개들의 전쟁,롯데엔터테인먼트,액션,96,청소년 관람불가,조병옥,NaN,0,91,2,23398.0,train,2012,11
1,내부자들,(주)쇼박스,느와르,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501.0,train,2015,11
2,은밀하게 위대하게,(주)쇼박스,액션,123,15세 관람가,장철수,220775.25,4,343,4,6959083.0,train,2013,6


In [34]:
# 장르 라벨 인코딩
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(total.genre)
total.genre = encoder.transform(total.genre)
total.head()

,title,distributor,genre,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,from,year,month
0,개들의 전쟁,롯데엔터테인먼트,10,96,청소년 관람불가,조병옥,NaN,0,91,2,23398.0,train,2012,11
1,내부자들,(주)쇼박스,2,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501.0,train,2015,11
2,은밀하게 위대하게,(주)쇼박스,10,123,15세 관람가,장철수,220775.25,4,343,4,6959083.0,train,2013,6
3,나는 공무원이다,(주)NEW,11,101,전체 관람가,구자홍,23894.00,2,20,6,217866.0,train,2012,7
4,불량남녀,쇼박스(주)미디어플렉스,11,108,15세 관람가,신근호,1.00,1,251,2,483387.0,train,2010,11


In [35]:
# 관람등급 라벨 인코딩
rat_encoder = LabelEncoder()
rat_encoder.fit(total.screening_rat)
total.screening_rat = rat_encoder.transform(total.screening_rat)
total.head()

,title,distributor,genre,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,from,year,month
0,개들의 전쟁,롯데엔터테인먼트,10,96,3,조병옥,NaN,0,91,2,23398.0,train,2012,11
1,내부자들,(주)쇼박스,2,130,3,우민호,1161602.50,2,387,3,7072501.0,train,2015,11
2,은밀하게 위대하게,(주)쇼박스,10,123,1,장철수,220775.25,4,343,4,6959083.0,train,2013,6
3,나는 공무원이다,(주)NEW,11,101,2,구자홍,23894.00,2,20,6,217866.0,train,2012,7
4,불량남녀,쇼박스(주)미디어플렉스,11,108,1,신근호,1.00,1,251,2,483387.0,train,2010,11


In [36]:
# 상위 6개의 배급사를 제외하고 기타로 처리
distributor_list = total.distributor.value_counts()[:6]
def func(distributor):
  if distributor in distributor_list:
    return distributor
  else:
    return '기타'

total['distributor'] = total['distributor'].apply(lambda x : func(x))

In [37]:
# 배급사 라벨 인코딩
dis_encoder = LabelEncoder()
dis_encoder.fit(total['distributor'])
total['distributor'] = dis_encoder.transform(total['distributor'])
total.head()

,title,distributor,genre,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,from,year,month
0,개들의 전쟁,5,10,96,3,조병옥,NaN,0,91,2,23398.0,train,2012,11
1,내부자들,2,2,130,3,우민호,1161602.50,2,387,3,7072501.0,train,2015,11
2,은밀하게 위대하게,2,10,123,1,장철수,220775.25,4,343,4,6959083.0,train,2013,6
3,나는 공무원이다,0,11,101,2,구자홍,23894.00,2,20,6,217866.0,train,2012,7
4,불량남녀,4,11,108,1,신근호,1.00,1,251,2,483387.0,train,2010,11


In [38]:
dir = total[:600].groupby('director', as_index=False)[['director','box_off_num']].mean().sort_values(by='box_off_num',ascending=False)
dir

,director,box_off_num
264,윤제균,14262766.0
416,최동훈,12845252.0
241,양우석,11374879.0
201,봉준호,9350351.0
448,한재림,9135806.0
...,...,...
411,진광교,36.0
289,이세일,10.0
10,고구마,8.0
85,김정호,2.0


In [39]:
# 감독 그룹화
top10_list = dir['director'][:10].to_list()
top20_list = dir['director'][10:20].to_list()
top50_list = dir['director'][20:50].to_list()
top100_list = dir['director'][50:100].to_list()
top200_list = dir['director'][100:200].to_list()
bottom_list = dir['director'][200:].to_list()
def func(director):
  if director in top10_list:
    return 'top10_list'
  elif director in top20_list:
    return 'top20_list'
  elif director in top50_list:
    return 'top50_list'
  elif director in top100_list:
    return 'top100_list'
  elif director in top200_list:
    return 'top200_list'
  else:
    return 'bottom_list'
total['director'] = total['director'].apply(lambda x : func(x))
total['director'].unique()

array(['bottom_list', 'top50_list', 'top10_list', 'top200_list',
       'top100_list', 'top20_list'], dtype=object)

In [40]:
# 감독 라벨 인코딩
dir_encoder = LabelEncoder()
dir_encoder.fit(total['director'])
total['director'] = dir_encoder.transform(total['director'])
total.head()

,title,distributor,genre,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,from,year,month
0,개들의 전쟁,5,10,96,3,0,NaN,0,91,2,23398.0,train,2012,11
1,내부자들,2,2,130,3,5,1161602.50,2,387,3,7072501.0,train,2015,11
2,은밀하게 위대하게,2,10,123,1,2,220775.25,4,343,4,6959083.0,train,2013,6
3,나는 공무원이다,0,11,101,2,3,23894.00,2,20,6,217866.0,train,2012,7
4,불량남녀,4,11,108,1,3,1.00,1,251,2,483387.0,train,2010,11


In [41]:
# dir_prev_bfnum, title 칼럼 drop
total.drop(columns=['dir_prev_bfnum', 'title'], inplace=True)
total.head(3)

,distributor,genre,time,screening_rat,director,dir_prev_num,num_staff,num_actor,box_off_num,from,year,month
0,5,10,96,3,0,0,91,2,23398.0,train,2012,11
1,2,2,130,3,5,2,387,3,7072501.0,train,2015,11
2,2,10,123,1,2,4,343,4,6959083.0,train,2013,6


In [42]:
total = total.drop('from', axis=1)

In [15]:
total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 843 entries, 0 to 242
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   distributor    843 non-null    int64  
 1   genre          843 non-null    int64  
 2   time           843 non-null    int64  
 3   screening_rat  843 non-null    int64  
 4   director       843 non-null    int64  
 5   dir_prev_num   843 non-null    int64  
 6   num_staff      843 non-null    int64  
 7   num_actor      843 non-null    int64  
 8   box_off_num    600 non-null    float64
 9   year           843 non-null    int64  
 10  month          843 non-null    int64  
dtypes: float64(1), int64(10)
memory usage: 79.0 KB


In [43]:
train = total[:600]
test = total[600:]

In [17]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 600 entries, 0 to 599
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   distributor    600 non-null    int64  
 1   genre          600 non-null    int64  
 2   time           600 non-null    int64  
 3   screening_rat  600 non-null    int64  
 4   director       600 non-null    int64  
 5   dir_prev_num   600 non-null    int64  
 6   num_staff      600 non-null    int64  
 7   num_actor      600 non-null    int64  
 8   box_off_num    600 non-null    float64
 9   year           600 non-null    int64  
 10  month          600 non-null    int64  
dtypes: float64(1), int64(10)
memory usage: 56.2 KB


In [44]:
# train_test_split
from sklearn.model_selection import train_test_split

X = train.drop('box_off_num', axis=1)
y = train['box_off_num']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [45]:
# XGBRegressor
from xgboost import XGBRegressor

xgb = XGBRegressor(random_state=0)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)

np.sqrt(mean_squared_error(y_test,xgb_pred))

[12:37:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


860446.7108542774

In [46]:
from sklearn.model_selection import GridSearchCV
params = {'max_depth':[4,6,8,10],
          'min_child_weight':[0.5,1,1.5],
          'subsample':[0.6,0.8,1],
          'gamma':[0,0.05,0.1,0.15,0.2]
}

grid_clf = GridSearchCV(xgb, param_grid=params, scoring='neg_mean_squared_error', cv=3)
grid_clf.fit(X_train, y_train)
print('최적의 하이퍼 파라미터 : {0}, 최적 평균 Negative MSE scores : {1:.3f}'.format(grid_clf.best_params_,
                                                     np.sqrt(-1 * grid_clf.best_score_)))

[12:37:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:37:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:37:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:37:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:37:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:37:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:37:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:37:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [47]:
# XGBRegressor
from xgboost import XGBRegressor

xgb_final = XGBRegressor(gamma=0, max_depth=6, min_child_weight=0.5, subsample=0.8 ,random_state=0)
xgb_final.fit(X_train, y_train)
xgb_final_pred = xgb_final.predict(X_test)

np.sqrt(mean_squared_error(y_test,xgb_final_pred))

[12:38:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


863238.5456175079

In [51]:
train = pd.read_csv("movies_train.csv")
test = pd.read_csv("movies_test.csv")
# train, test 합치기
train['from'] = 'train'
test['from'] = 'test'
total = train.append(test)
# 개봉날짜 형변환 또는 연월 분리
total['year'] = total['release_time'].apply(lambda x: int(x[:4]))
total['month'] = total['release_time'].apply(lambda x: int(x[5:7]))
total =  total.drop(['release_time'],axis = 1)
# 장르 라벨 인코딩
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(total.genre)
total.genre = encoder.transform(total.genre)
# 관람 등급 라벨 인코딩
rat_encoder = LabelEncoder()
rat_encoder.fit(total.screening_rat)
total.screening_rat = rat_encoder.transform(total.screening_rat)
dist = total[:600].groupby('distributor', as_index=False)[['distributor','box_off_num']].mean().sort_values(by='box_off_num',ascending=False)
dist
# 배급사 그룹화
top10_list = dist['distributor'][:10].to_list()
top20_list = dist['distributor'][10:20].to_list()
top50_list = dist['distributor'][20:50].to_list()
top100_list = dist['distributor'][50:100].to_list()
top200_list = dist['distributor'][100:].to_list()
def func(distributor):
  if distributor in top10_list:
    return 'top10_list'
  elif distributor in top20_list:
    return 'top20_list'
  elif distributor in top50_list:
    return 'top50_list'
  elif distributor in top100_list:
    return 'top100_list'
  else:
    return 'bottom_list'
total['distributor'] = total['distributor'].apply(lambda x : func(x))
total['distributor'].unique()
# 배급사 라벨 인코딩
dis_encoder = LabelEncoder()
dis_encoder.fit(total['distributor'])
total['distributor'] = dis_encoder.transform(total['distributor'])
# 감독 그룹화
top10_list = dir['director'][:10].to_list()
top20_list = dir['director'][10:20].to_list()
top50_list = dir['director'][20:50].to_list()
top100_list = dir['director'][50:100].to_list()
top200_list = dir['director'][100:200].to_list()
bottom_list = dir['director'][200:].to_list()
def func(director):
  if director in top10_list:
    return 'top10_list'
  elif director in top20_list:
    return 'top20_list'
  elif director in top50_list:
    return 'top50_list'
  elif director in top100_list:
    return 'top100_list'
  elif director in top200_list:
    return 'top200_list'
  else:
    return 'bottom_list'
total['director'] = total['director'].apply(lambda x : func(x))
# 감독 라벨 인코딩
dir_encoder = LabelEncoder()
dir_encoder.fit(total['director'])
total['director'] = dir_encoder.transform(total['director'])
# dir_prev_bfnum, title 칼럼 drop
total.drop(columns=['dir_prev_bfnum', 'title'], inplace=True)
total = total.drop('from', axis=1)
train = total[:600]
test = total[600:]

In [52]:
test_x = test.drop('box_off_num', axis=1)
pred = xgb_final.predict(test_x)
test_x = pd.DataFrame(pred)
submission['box_off_num'] = test_x

submission.to_csv('xgb15.csv', encoding = 'utf-8-sig', index=False)